In [1]:
%matplotlib inline

In [2]:
from collections import defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from networkx.algorithms.approximation import clique
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm

from utils import Datasets_Data, RFE_custom, load_or_calc_and_save, ytrue_to_partition, calc_avranks

In [3]:
data_hub = Datasets_Data()
results, results_modularity_any3, modularity_results = data_hub.load_precalculated()
X, y, X_train, y_train, X_val, y_val, feature_names = data_hub.make_dataset(results_modularity_any3)

100%|██████████| 24/24 [00:00<00:00, 1798.43it/s]

wrapper: cache file ../../cache/cache/feature_importance/cora_DB.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_EC.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_HA.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_HCI.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_IR.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_Net.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/dolphins.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/eu-core.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/eurosis.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/football.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance

# Feature importance

In [4]:
X_train[0]

array([ 1.13876919, -0.49987918, -0.71082348,  0.05246037, -0.30272497,
        7.49358751,  3.8630649 ,  1.15514418, -1.35364409,  0.79303828,
        0.83106491,  0.81310777, -0.14811308,  1.1681215 ,  1.01302749,
        1.27660159, -0.69948906, -0.70235936,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [5]:
estimator = LogisticRegression(max_iter=10000)
estimator.fit(X_train, y_train)
pd.DataFrame([{'feature': k, 'importance': v} for k, v in sorted(zip(feature_names, estimator.coef_[0]), key=lambda x: -np.abs(x[1]))])

,feature,importance
0,median_sp,-3.193033
1,density,-1.179002
2,kernel_FE,1.116905
3,kernel_logFor,0.959298
4,kernel_RSP,0.959298
5,median_deg,-0.942677
6,avg(deg | deg > avg_deg),-0.808434
7,log(n)/k * p_in/p_out,-0.787813
8,avg_sp,0.784672
9,kernel_Comm,-0.765955


In [ ]:
estimator = LogisticRegression(max_iter=10000)
selector = RFE_custom(estimator)  # , n_features_to_select=3, verbose=1
selector = selector.fit(X_train, y_train, X_val, y_val, feature_names)

  0%|          | 0/18 [00:00<?, ?it/s]

all features, acc=0.934, f1=0.639


  0%|          | 0/153 [00:00<?, ?it/s]

1 features, set=('median_deg',) acc=0.932, f1=0.597


  0%|          | 0/816 [00:00<?, ?it/s]

2 features, set=('log(n)/k * p_in/p_out', 'n/k * p_in/p_out') acc=0.932, f1=0.633


 10%|█         | 82/816 [00:25<04:52,  2.51it/s]

In [ ]:
pd.DataFrame(zip(feature_names, selector.support_, selector.ranking_), columns=['feature', 'to choose', 'rank']).sort_values('rank')

In [ ]:
estimator = LogisticRegression()
estimator.fit(X_train[:, selector.support_], y_train)

In [ ]:
print('\n'.join([f'{k}\t{v:.3f}' for k, v in sorted(zip(np.array(feature_names)[selector.support_], estimator.coef_[0]), key=lambda x: -np.abs(x[1]))]))

In [ ]:
y_pred = estimator.predict(X_val[:, selector.support_])

In [ ]:
accuracy_score(y_val.ravel(), y_pred.ravel())

In [ ]:
f1_score(y_val.ravel(), y_pred.ravel())